# Alerts Generator

This notebook was made to facilitate the implementation of chemical alerts in python code as functions to calculate alerts-fingerprint

In [21]:
import pandas as pd
import numpy as np

## 1. Define output name

In [22]:
out_name = 'SYGMA_REACTION_RULES'

## 2. Import ruleset file:

This file must be an excel files with 3 different columns with the header: 

- **name**: That column contain the name of alerts
- **smarts**: That column contain every smarts you want to implement
- **description**: That column contain a brief description of alerts

In [45]:
ruleset_file_name = "SyGMA_rulesClear.xlsx"

In [46]:
data = pd.read_excel(ruleset_file_name)

In [47]:
data

,name,description,smarts
0,REACTION_N_dealkylation_1,'N-demethylation_(R-NHCH3)',[*;!c:1][NH1;X3:2][CH3]
1,REACTION_N_dealkylation_2,'N-demethylation_(c-NHCH3)',[c:1][NH1;X3:2][CH3]
2,REACTION_N_dealkylation_3,'N-demethylation_(R-N(CH3)2)',[*;!c:1][NH0;X3:2]([CH3])[CH3:3]
3,REACTION_N_dealkylation_4,'N-demethylation_(c-N(CH3)2)',[c:1][NH0;X3:2]([CH3])[CH3:3]
4,REACTION_N_dealkylation_5,'N-demethylation_(R-N(CR)CH3)',[*;!$([CH3]):1][NH0;X3:2]([CH3])[#6;!$([CH3]):3]
...,...,...,...
164,REACTION_special_rules_13,'oxidation_(C=N)',[NX2:1]=[CH1:2]
165,REACTION_special_rules_14,'deiodonidation',[#6X3:1][I]
166,REACTION_special_rules_15,'nitrile_to_amide',[C:1]#N
167,REACTION_steroids_1,'steroid_d5d4',[C;$(C~1~C~C~C~C~2~C~C~C~3~C~4~C~C~C~C~4~C~C~C...


### Esecution code:

In [48]:
def build_alert(data):
    alert_list = []
    for row in range(len(data)):
        t = f"def {data.loc[row,['name']][0]}(molecule: Chem.rdchem.Mol):\n\t#{data.loc[row,['description']][0]}\n\tsubstructure = '{data.loc[row,['smarts']][0]}'\n\tsubstructure = Chem.MolFromSmarts(substructure)\n\tindices = list(molecule.GetSubstructMatches(substructure))\n\tif indices==[]: out=0\n\telse: out=1\n\treturn out\n\n"
       
        if row == 0: txt = f"import pandas as pd\nfrom rdkit import Chem\nimport numpy as np\n\n"+t
        else: txt = txt+t
            
    return txt

In [50]:
def descriptor(data):
    for row in range(len(data)):
        if row == 0: 
            txt = f"def descriptor(mol: Chem.rdchem.Mol):\n\n\tSA_{data.loc[row,['name']][0]} = []\n\t"
            txt2 = f"lst_chemical = [SA_{data.loc[row,['name']][0]},\n\t\t"
            txt3 = f"if mol != 'none':\n\t\tSA_{data.loc[row,['name']][0]}.append({data.loc[row,['name']][0]}(mol))\n\t\t"
            txt5 = f"out = pd.DataFrame(zip(SA_{data.loc[row,['name']][0]},"
            txt6 = f"columns = ['{data.loc[row,['name']][0]}',"
        elif row < len(data)-1:
            txt = txt + f"SA_{data.loc[row,['name']][0]} = []\n\t"
            txt2 = txt2 + f"SA_{data.loc[row,['name']][0]},\n\t\t"
            txt3 = txt3 + f"SA_{data.loc[row,['name']][0]}.append({data.loc[row,['name']][0]}(mol))\n\t\t"
            txt5 = txt5 + f"SA_{data.loc[row,['name']][0]},"
            txt6 = txt6 + f"'{data.loc[row,['name']][0]}',"
        else: 
            txt = txt + f"SA_{data.loc[row,['name']][0]} = []\n\t"
            txt2 = txt2 + f"SA_{data.loc[row,['name']][0]}]\n\t"
            txt3 = txt3 + f"SA_{data.loc[row,['name']][0]}.append({data.loc[row,['name']][0]}(mol))\n\t"
            txt5 = txt5 + f"SA_{data.loc[row,['name']][0]}),"
            txt6 = txt6 + f"'{data.loc[row,['name']][0]}'])\n\t"
    txt4 = f"else:\n\t\tfor chem_group in lst_chemical:\n\t\t\tchem_group.append('none')"
    txt = txt + "\n\t" + txt2 + "\n\t" + txt3 +"\n\t"+ txt4 + "\n\t" + txt5 +"\n" + txt6 + "\n\treturn out"
    return txt


a = build_alert(data)
d = descriptor(data)
txt = a + "\n\n" + "\n\n" + d
with open(f"{out_name}.txt", "w") as f:f.write(txt)